# ** Predicting Elemental Concentrations in Scrap Alloys Using PLSR and Spectral Data **

This script processes spectral data files, extracts peak intensities of specific elemental emission lines, and uses Partial Least Squares Regression (PLSR) to predict elemental concentrations in unknown test samples.

⚛How it works
1. Data Loading and Processing
Recursively loads .txt spectral files from specified training and test directories.

- Each file contains wavelength and intensity data.

- Applies baseline correction and Standard Normal Variate (SNV) normalization to reduce noise and standardize spectra.

- Averages multiple measurements per sample to obtain a representative spectrum.

- Extracts peak intensity values near predefined emission wavelengths for elements: Mn, Si, Mg, Cu, and Zn.

2. Training Data Preparation
- Matches processed spectral peaks with known reference elemental concentrations for training samples.

3. PLSR Model Training and Evaluation
- For each element, selects relevant emission line features present in both training and test sets.

- Fits a PLS regression model on the training set.

- Evaluates model fit on training data using R² and RMSE metrics.

4. Prediction on Test Data
- Uses trained models to predict elemental concentrations for test samples.

5. Visualization and Output
- Displays predicted concentrations in a table with sample IDs.

- Generates grouped bar charts comparing predicted elemental concentrations across test samples.

- Plots histograms to visualize the distribution of predicted concentrations per element.

- Creates heatmaps for a compact overview of predictions.

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.cross_decomposition import PLSRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
import natsort

# --- Configuration ---
TRAIN_DIR = "C:/Users/KA/OneDrive/Desktop/BAMAN/28.01.2025 BAM-Alu"👈#Certified reference materials 
TEST_DIR = "C:/Users/KA/OneDrive/Desktop/T8/"👈#Scrap alloys with unknown concentrations

peak_lines = {
    "Mn": [403.08, 403.31],
    "Si": [251.61, 288.16],
    "Mg": [285.21, 383.23, 383.83, 518.36],
    "Cu": [324.75, 327.40],
    "Zn": [334.50, 468.01, 481.05]
}

reference_concentrations = {
    'BAM-308':     {'Mn': 0.0342, 'Si': 0.0707, 'Mg': 2.2900, 'Cu': 1.3150, 'Zn': 5.6700}, 
    'BAM-311':     {'Mn': 0.694,  'Si': 0.204,  'Mg': 1.567,  'Cu': 4.653, 'Zn': 0.2005},
    'BAM-M308a':   {'Mn': 0.0343, 'Si': 0.072,  'Mg': 2.280,  'Cu': 1.360, 'Zn': 5.61},
    'BAM-M318':    {'Mn': 0.0985, 'Si': 1.211,  'Mg': 0.356,  'Cu': 0.0908, 'Zn': 0.0486},
    'ERM-EB313':   {'Mn': 0.4950, 'Si': 0.363,  'Mg': 3.400,  'Cu': 0.0931, 'Zn': 0.1580},
    'ERM-EB314a':  {'Mn': 0.404,  'Si': 11.51,  'Mg': 0.196,  'Cu': 2.080, 'Zn': 1.1},
    'ERM-EB315a':  {'Mn': 0.311,  'Si': 9.88,   'Mg': 0.446,  'Cu': 2.460, 'Zn': 0.801},
    'ERM-EB317':   {'Mn': 0.0912, 'Si': 0.0271, 'Mg': 2.390,  'Cu': 1.770, 'Zn': 6.93}
}

# --- Helper Functions ---
def load_raw_data(directory):
    data = {}
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.txt'):
                sample = os.path.basename(os.path.dirname(os.path.join(root, file)))
                try:
                    df = pd.read_csv(os.path.join(root, file), delimiter=';', names=['wavelength', 'intensity'])
                    df['wavelength'] = pd.to_numeric(df['wavelength'], errors='coerce')
                    df.dropna(subset=['wavelength'], inplace=True)
                    data.setdefault(sample, []).append(df)
                except:
                    continue
    return data

def baseline_correction(df):
    from pybaselines import whittaker as pl
    for col in [c for c in df.columns if c != 'wavelength']:
        baseline, _ = pl.asls(df[col].values)
        df[col] = (df[col] - baseline).clip(lower=0)
    return df

def snv(df):
    for col in [c for c in df.columns if c != 'wavelength']:
        spectrum = df[col]
        mean = spectrum.mean()
        std = spectrum.std()
        df[col] = (spectrum - mean) / std if std != 0 else 0
    return df

def average_measurements(dfs):
    combined = pd.concat(dfs, axis=1)
    combined = combined.loc[:, ~combined.columns.duplicated()]
    averaged = combined.drop(columns='wavelength', errors='ignore').mean(axis=1)
    averaged_df = pd.DataFrame({'wavelength': dfs[0]['wavelength'], 'intensity': averaged})
    return averaged_df

def extract_peak_areas(df):
    peaks = {}
    for element, wls in peak_lines.items():
        for wl in wls:
            region = df[(df['wavelength'] >= wl - 0.1) & (df['wavelength'] <= wl + 0.1)]
            peak_value = region['intensity'].max() if not region.empty else np.nan
            peaks[f"{element}_{wl:.2f}"] = peak_value
    return peaks

def process_samples(path):
    raw_data = load_raw_data(path)
    peak_data = {}
    for sample, dfs in raw_data.items():
        processed = [snv(baseline_correction(df.copy())) for df in dfs]
        avg = average_measurements(processed)
        peaks = extract_peak_areas(avg)
        peak_data[sample] = peaks
    df = pd.DataFrame.from_dict(peak_data, orient='index')
    df.index.name = 'Sample'
    return df.sort_index(key=natsort.natsort_keygen())

# --- Process training and test sets ---
steel_df = process_samples(TRAIN_DIR)
test_df = process_samples(TEST_DIR)
y_train_df = pd.DataFrame.from_dict(reference_concentrations, orient='index')
steel_df = steel_df.loc[y_train_df.index]

# --- Model Training and Prediction ---
predictions_clipped = {}
predictions_raw = {}

for element in y_train_df.columns:
    feature_cols = [col for col in steel_df.columns if col.startswith(f"{element}_") and col in test_df.columns]
    if not feature_cols:
        print(f"Skipping {element}: no matching peaks in test data.")
        continue

    X_train = steel_df[feature_cols]
    y_train = y_train_df[element]
    X_test = test_df[feature_cols]

    model = make_pipeline(StandardScaler(), PLSRegression(n_components=min(3, len(feature_cols))))
    model.fit(X_train, y_train)

    # Evaluate on training set (raw only)
    y_train_pred = model.predict(X_train).flatten()
    r2 = r2_score(y_train, y_train_pred)
    rmse = mean_squared_error(y_train, y_train_pred, squared=False)
    print(f"\nElement: {element}")
    print(f"  R² (Train): {r2:.3f}")
    print(f"  RMSE (Train): {rmse:.4f}")

    # Predict test set
    y_pred_raw = model.predict(X_test).flatten()
    y_pred_clipped = np.clip(y_pred_raw, 0, None)

    predictions_raw[element] = y_pred_raw
    predictions_clipped[element] = y_pred_clipped

# --- Final Prediction Table ---
y_pred_df = pd.DataFrame(predictions_clipped, index=test_df.index)
y_pred_df.index = [f"T8_S{i+1}" for i in range(len(y_pred_df))]

print("\nPredicted concentrations for T8 test samples (clipped):")
print(y_pred_df.round(4).to_string())

# --- Grouped Bar Chart ---
ax = y_pred_df.plot(kind='bar', figsize=(7, 4), colormap='tab10')
plt.ylabel("Predicted Concentration (%)")
plt.title("Predicted Elemental Concentrations for T8 Unknown Material")
plt.xticks(rotation=45, ha='right')
plt.grid(True)
plt.tight_layout()
plt.show()

# --- Histograms ---
for col in y_pred_df.columns:
    plt.figure()
    sns.histplot(y_pred_df[col], kde=True)
    plt.title(f"Distribution of Predicted {col} Concentration")
    plt.xlabel(f"{col} Concentration (%)")
    plt.grid(True)
    plt.tight_layout()
    plt.close()

# --- Heatmap ---
plt.figure(figsize=(7, 5))
sns.heatmap(y_pred_df, annot=True, fmt=".2f", cmap="YlGnBu", cbar_kws={'label': 'Predicted Concentration (%)'})
plt.title("Predicted Elemental Concentrations for T8 Unknown Material")
plt.tight_layout()
plt.show()